# ReverseNumber Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread

In [3]:
#USER_JWT_PATH = "/home/staverm/workspace/coflows-dev/coflows/jwts.txt"
#user_jwt = open(USER_JWT_PATH).readline().strip()
#cl = CoLink("http://127.0.0.1:2021", user_jwt)

In [4]:
cl = colink_utils.start_colink_server()

# Start a few default workers

In [5]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_type: reverse_number_sequential
flow_id: 7513bdfd-8828-4424-8a77-99015470ca08
message_paths: ['push_tasks:e2022a70-86b2-47cb-9ba7-d3ace83c276e:msg']
parallel_dispatch: False

Called ReverseNumberSequential 
 state {'current_call': 'first_reverse_flow'}

~~~ Dispatch task ~~~
flow_type: reverse_number_sequential
flow_id: 7513bdfd-8828-4424-8a77-99015470ca08
message_paths: ['push_tasks:555e933f-7c9d-4aff-b868-8aba198f016e:msg']
parallel_dispatch: False

Called ReverseNumberSequential 
 state {'current_call': 'second_reverse_flow', 'initial_message': FlowMessage(message_id='b7461140-da9d-4ef9-b960-fee6542c792e', created_at='2024-03-16 03:48:24.870232531', created_by='Proxy_reverse_number_sequential', message_type='FlowMessage', data={'id': 0, 'number': 1234}, private_keys=[])}

~~~ Dispatch task ~~~
flow_type: reverse_number_sequential
flow_id: 7513bdfd-8828-4424-8a77-99015470ca08
mess

In [6]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_type: reverse_number_atomic
flow_id: 1eab578d-13da-44ec-a2e6-3880f01c8b64
message_paths: ['push_tasks:08b6b972-ab15-4e05-86c6-2d1ba32b8f79:msg']
parallel_dispatch: False

Called ReverseNumberAtomic 
 state {}

~~~ Dispatch task ~~~
flow_type: reverse_number_atomic
flow_id: 1eab578d-13da-44ec-a2e6-3880f01c8b64
message_paths: ['push_tasks:2ecab072-aa74-43ba-80e6-06b57d7c42d0:msg']
parallel_dispatch: False

Called ReverseNumberAtomic 
 state {}


# Serve

In [7]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic",
    singleton=True,
)
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential"
)

Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: True
Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

## Get instance

In [8]:
flow = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="reverse_number_sequential",
    user_id="local",
)

Mounted 1eab578d-13da-44ec-a2e6-3880f01c8b64 at flows:reverse_number_atomic:mounts:local:1eab578d-13da-44ec-a2e6-3880f01c8b64
Fetched singleton 1eab578d-13da-44ec-a2e6-3880f01c8b64
Mounted 7513bdfd-8828-4424-8a77-99015470ca08 at flows:reverse_number_sequential:mounts:local:7513bdfd-8828-4424-8a77-99015470ca08


In [9]:
input_data = {"id": 0, "number": 1234}
    
input_message = flow.package_input_message(input_data)
future = flow.get_reply_future(input_message)
reply_data = future.get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

Data sent:
 {'id': 0, 'number': 1234} 

REPLY:
 {'output_number': 1234} 



## Observe memory

In [10]:
def status():
    colink_utils.print_served_flows(cl, print_values=False)
    print("\nAll flow instances:")
    colink_utils.print_flow_instances(cl)
status()

 reverse_number_sequential/
   flow_class_name
   mounts/
     local/
       7513bdfd-8828-4424-8a77-99015470ca08/
         init
         config_overrides
         state
   default_dispatch_point
   singleton
   parallel_dispatch
   init
 reverse_number_atomic/
   parallel_dispatch
   init
   mounts/
     local/
       1eab578d-13da-44ec-a2e6-3880f01c8b64/
         state
         config_overrides
         init
   flow_class_name
   singleton
   default_dispatch_point

All flow instances:
 7513bdfd-8828-4424-8a77-99015470ca08
 1eab578d-13da-44ec-a2e6-3880f01c8b64
